# Segmenting and Clustering Neighborhoods in Toronto 
## Part I
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
![dataframe](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1564704000000&hmac=uEVxJTTRbXL7VZOoCSjIH7wHCGYf9f0ywQFzqFOYfQI)

3. To create the above dataframe:

    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table.
    - If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. So for the **9th** cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be **Queen's Park**.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.
    - Submit a link to your Notebook on your Github repository. 

In [1]:
# import the libraries
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
from pprint import pprint

In [2]:
# scrape the data from the page
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_page = requests.get(link).text
# pprint(raw_page)

In [3]:
## use beautiful soup to parse page
soup = BeautifulSoup(raw_page, "xml")
# pprint(soup)

In [4]:
## extract the table 
table = soup.find("table")
df = pd.read_html(str(table))[0]    
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# Cleaning the data according to the requirements

## Only process the cells that have an assigned borough. 
## Ignore cells with a borough that is Not assigned.
df = df[df["Borough"]!= "Not assigned"]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
## More than one neighborhood can exist in one postal code area. 
## For example, in the table on the Wikipedia page, 
## you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
## These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
groupby = df.groupby(["Postcode"])
clean_df = pd.DataFrame(columns=["Postcode", "Borough", "Neighbourhood"])

for postcode, every_group in groupby:
    list_neighbourhood = every_group["Neighbourhood"].tolist()
    str_neighbourhood  = ",".join(list_neighbourhood)
    new_line = pd.DataFrame({"Postcode" : postcode,
                      "Borough" : every_group["Borough"].iloc[0], 
                      "Neighbourhood" : str_neighbourhood}, index=[0])
    
    clean_df = clean_df.append(new_line, ignore_index=True)

clean_df.head()
    

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
## So for the 9th cell in the table on the Wikipedia page, 
## the value of the Borough and the Neighborhood columns will be Queen's Park.

for index, row in clean_df.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]

In [8]:
clean_df.shape

(103, 3)

## Part II

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

```python
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [9]:
## get all geographical coordinates of the neighborhoods.
geo_data_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_data_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
## Merge the coordinates info in Postalcode/Borough/Neighborhood dataframe.
merged_df = pd.merge(clean_df, geo_data_df,left_on='Postcode',right_on='Postal Code',how='left').drop('Postal Code', axis=1)
merged_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part III

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

### Create a map of Toronto

In [11]:
import folium

In [12]:
toronto_latitude = 43.6532
toronto_lontitude = -79.3832
toronto_map = folium.Map(location=[toronto_latitude, toronto_lontitude],
                     zoom_start=10)
toronto_map 

In [13]:
for lat, lng, brh, nbh in zip(merged_df["Latitude"],
                             merged_df["Longitude"],
                             merged_df["Borough"],
                             merged_df["Neighbourhood"]):
    label = f"{nbh},{brh}"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color="green",
    fill=True,
    ).add_to(toronto_map)
    
toronto_map   